# Tabular data

[video](https://www.youtube.com/watch?v=9YK6AnqpuEA),
[documentation](https://docs.fast.ai/tabular.html),
[notebook](https://github.com/fastai/course-v3/blob/master/nbs/dl1/lesson4-tabular.ipynb)

Before random forest, now neural nets.


### Time-series tabular data
Add columns like, is a weekday? is on holiday?
Not commonly used RNNs

In [23]:
# Temporal fix to a pandas bug
get_ipython().config.get('IPKernelApp', {})['parent_appname'] = ""

In [2]:
from fastai.tabular import * 

In [11]:
path = untar_data(URLs.ADULT_SAMPLE)
df = pd.read_csv(path/'adult.csv')
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,49,Private,101320,Assoc-acdm,12.0,Married-civ-spouse,NaN,Wife,White,Female,0,1902,40,United-States,>=50k
1,44,Private,236746,Masters,14.0,Divorced,Exec-managerial,Not-in-family,White,Male,10520,0,45,United-States,>=50k
2,38,Private,96185,HS-grad,NaN,Divorced,NaN,Unmarried,Black,Female,0,0,32,United-States,<50k
3,38,Self-emp-inc,112847,Prof-school,15.0,Married-civ-spouse,Prof-specialty,Husband,Asian-Pac-Islander,Male,0,0,40,United-States,>=50k
4,42,Self-emp-not-inc,82297,7th-8th,NaN,Married-civ-spouse,Other-service,Wife,Black,Female,0,0,50,United-States,<50k


## Variables
- **Categorical variables**: We will use embeddings.
- **Continuos variables**: We will feed them directly.
- **Dependent variable**: is the label.

> Note that Pandas can create embeddings but we will relay on the fastai data api.

In [19]:
categor_vars = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race']
contin_vars  = ['age', 'fnlwgt', 'education-num']
depend_var   = 'salary'

## Preprocess
Like transformations but you only do it once at the begining.

- **FillMissing**: Fill the missing values in continuous columns.
- **Categorify**: Transform the categorical variables to that type.
- **Normalize**: Normalize the continuous variables.

In [16]:
procs = [FillMissing, Categorify, Normalize]

## Get data

In [17]:
test = TabularList.from_df(df.iloc[800:1000].copy(), path=path, cat_names=categor_vars, cont_names=contin_vars)

data = (TabularList.from_df(df, path=path, cat_names=categor_vars, cont_names=contin_vars, procs=procs)
                           .split_by_idx(list(range(800,1000)))
                           .label_from_df(cols=depend_var)
                           .add_test(test)
                           .databunch())

In [18]:
data.show_batch(rows=10)

workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num,target
Local-gov,Some-college,Married-civ-spouse,Craft-repair,Husband,White,False,-0.3362,0.1938,-0.0312,<50k
Local-gov,Bachelors,Married-civ-spouse,Prof-specialty,Husband,White,False,-0.3362,-0.5062,1.1422,<50k
Self-emp-not-inc,Assoc-voc,Married-civ-spouse,Farming-fishing,Husband,White,False,0.1769,-0.7243,0.3599,>=50k
Private,11th,Never-married,Adm-clerical,Not-in-family,White,False,-1.0692,-0.0716,-1.2046,<50k
Private,Some-college,Married-civ-spouse,Prof-specialty,Husband,White,False,2.1559,-0.7783,-0.0312,<50k
Private,HS-grad,Never-married,Craft-repair,Not-in-family,Black,False,-0.9226,0.9535,-0.4224,<50k
Self-emp-not-inc,HS-grad,Never-married,Prof-specialty,Unmarried,White,False,-1.2158,0.1251,-0.4224,>=50k
Private,Some-college,Never-married,Other-service,Own-child,White,False,-1.4357,1.1069,-0.0312,<50k
Private,Assoc-acdm,Married-civ-spouse,Craft-repair,Husband,White,False,-1.0692,-0.1477,0.7511,<50k
Private,HS-grad,Married-civ-spouse,Machine-op-inspct,Husband,White,False,0.6899,-0.1298,-0.4224,<50k


## Train

In [20]:
learn = tabular_learner(data, layers=[200,100], metrics=accuracy)

In [21]:
learn.fit(1, 1e-2)

epoch,train_loss,valid_loss,accuracy
1,0.359662,0.425281,0.820000


## Test

In [22]:
row = df.iloc[0]
learn.predict(row)

(Category <50k, tensor(0), tensor([0.6453, 0.3547]))